In [1]:
import pandas as pd
import numpy as np
import time
import itertools
import datetime
import matplotlib.pyplot as plt
from datetime import *
import pandas_gbq
from datetime import datetime, timedelta, date
from pathlib import Path
from zipfile import ZipFile
import urllib
from PIL import Image
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
# Standard plotly imports
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import requests
import json
project_id = "perceptive-ivy-290216"

In [2]:
#Change the date to the previous day
query = """
SELECT
*
FROM
`perceptive-ivy-290216.f1_api.race_lap_time`
UNION ALL
SELECT
*
FROM
`perceptive-ivy-290216.f1_api.final_driver_positions_all`
"""
project_id = "perceptive-ivy-290216"
df_bq = pandas_gbq.read_gbq(query, project_id=project_id, dialect='standard')

Downloading: 100%|██████████|


In [5]:
GP=df_bq[((df_bq["GP"]=="Mexico City Grand Prix") & (df_bq["Year"]==2024))]
GP["LapNumber"]=GP["LapNumber"].astype(int)
GP=GP.sort_values(['Driver','LapNumber'])
GP["DriverNumber"]=GP["DriverNumber"].astype(int)
year=GP['Year'].iloc[0]
gp=GP['GP'].iloc[0]
GP.head()

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_30161/764475036.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Time,Driver,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,Sector3Time,Sector1SessionTime,Sector2SessionTime,Sector3SessionTime,SpeedI1,SpeedI2,SpeedFL,SpeedST,IsPersonalBest,Compound,TyreLife,FreshTyre,Team,LapStartTime,LapStartDate,TrackStatus,Position,Deleted,DeletedReason,FastF1Generated,IsAccurate,Year,GP
22679,0 days 00:57:46.897000,ALB,23,NaT,1,1.0,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaN,NaN,NaN,NaN,false,MEDIUM,1.0,true,Williams,0 days 00:56:01.523000,NaT,124,9.0,False,None,True,false,2024,Mexico City Grand Prix
12245,None,ALB,23,None,72,NaN,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,None,None,NaN,None,None,None,None,None,19.0,<NA>,None,<NA>,None,2024,Mexico City Grand Prix
22675,0 days 00:58:07.614000,ALO,14,0 days 00:02:05.776000,1,1.0,NaT,NaT,NaT,0 days 00:00:58.712000,0 days 00:00:31.915000,NaT,0 days 00:57:35.756000,0 days 00:58:07.761000,267.0,70.0,145.0,294.0,false,MEDIUM,2.0,false,Aston Martin,0 days 00:56:01.523000,NaT,124,13.0,False,None,False,false,2024,Mexico City Grand Prix
21479,0 days 01:00:25.257000,ALO,14,0 days 00:02:17.643000,2,1.0,NaT,NaT,0 days 00:01:06.993000,0 days 00:00:41.044000,0 days 00:00:29.606000,0 days 00:59:14.608000,0 days 00:59:55.652000,0 days 01:00:25.258000,169.0,202.0,209.0,192.0,True,MEDIUM,3.0,False,Aston Martin,0 days 00:58:07.614000,NaT,4,12.0,False,,False,False,2024,Mexico City Grand Prix
21482,0 days 01:02:41.241000,ALO,14,0 days 00:02:15.984000,3,1.0,NaT,NaT,0 days 00:00:58.825000,0 days 00:00:46.891000,0 days 00:00:30.268000,0 days 01:01:24.083000,0 days 01:02:10.974000,0 days 01:02:41.242000,203.0,262.0,226.0,78.0,True,MEDIUM,4.0,False,Aston Martin,0 days 01:00:25.257000,NaT,4,12.0,False,,False,False,2024,Mexico City Grand Prix


In [6]:
fig=px.line(GP, x="LapNumber", y="Position", color='Driver',
#             markers=True,
        title="Race Position Ladder for the {} {}".format(year,gp),
        hover_data=['Compound','TyreLife','Team','LapTime','Stint', 'PitInTime', 'PitOutTime', 'Sector1Time', 'Sector2Time', 'LapStartTime', 'Year'],
        height=1000, width=1200, template="plotly_dark",
             color_discrete_map={
                 "ALB": "#64C4FF",
                 "ALO": "#229971",
                 "BEA": "#B6BABD",
                 "BOT": "#52e252",
                 "COL": "#37BEDD",
                 "GAS": "#0093cc",
                 "HAM": "#27F4D2",
                 "HUL": "#B6BABD",
                 "LAW": "#6692FF",
                 "LEC": "#E80020",
                 "MAG": "#B6BABD",
                 "NOR": "#FF8000",
                 "OCO": "#0093cc",
                 "PER": "#3671C6",
                 "PIA": "#FF8000",
                 "RIC": "#6692FF",
                 "RUS": "#27F4D2",
                 "SAI": "#E80020",
                 "SAR": "#37BEDD",
                 "STR": "#229971",
                 "TSU": "#6692FF",
                 "VER": "#3671C6",
                 "ZHO": "#52e252"
          },
        )

fig.update_layout(
    title_x=0.5,
    yaxis_range=[1, 21],
    margin=dict(l=10, r=5, t=55, b=20),
    font=dict(
        family="PT Sans Narrow",
        size=13,
        color="White"
    ),

    yaxis = dict(
    tickmode = 'linear',
    tick0 = 1,
    dtick = 1
    ),

    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),

    margin_pad=10
)
fig.update_yaxes(autorange="reversed")

# fig.update_layout(
    # updatemenus=[
    #     {
    #         "buttons": [
    #             {
    #                 "label": t.name,
    #                 "method": "restyle",
    #                 "args": [{"visible": [t2.name == t.name for t2 in fig.data]}],
    #             }
    #             for t in fig.data
    #         ]
    #     }
    # ]
# )

In [7]:
fig.write_html("/Users/rdesh723/statpulse-html/plots/Race Ladder/{}/{} {} Race Ladder.html".format(year,year,gp),full_html=False, include_plotlyjs='cdn')

In [ ]:
#MULTI GP RACE LADDER

In [10]:
#2024
# race_multi=["Bahrain Grand Prix", "Saudi Arabian Grand Prix", "Australian Grand Prix", "Japanese Grand Prix","Chinese Grand Prix","Miami Grand Prix","Emilia Romagna Grand Prix","Monaco Grand Prix","Canadian Grand Prix","Spanish Grand Prix","Austrian Grand Prix","British Grand Prix","Hungarian Grand Prix","Belgian Grand Prix"]
#2023
# race_multi=["Abu Dhabi Grand Prix","Australian Grand Prix","Austrian Grand Prix","Azerbaijan Grand Prix","Bahrain Grand Prix","Belgian Grand Prix","British Grand Prix","Canadian Grand Prix","Dutch Grand Prix","Hungarian Grand Prix","Italian Grand Prix","Japanese Grand Prix","Las Vegas Grand Prix","Mexico City Grand Prix","Miami Grand Prix","Monaco Grand Prix","Pre-Season Testing","Qatar Grand Prix","Saudi Arabian Grand Prix","Singapore Grand Prix","Spanish Grand Prix","São Paulo Grand Prix","United States Grand Prix"]
#2022
# race_multi=["Abu Dhabi Grand Prix","Australian Grand Prix","Austrian Grand Prix","Azerbaijan Grand Prix","Bahrain Grand Prix","Belgian Grand Prix","British Grand Prix","Canadian Grand Prix","Dutch Grand Prix","Emilia Romagna Grand Prix","French Grand Prix","Hungarian Grand Prix","Italian Grand Prix","Japanese Grand Prix","Mexico City Grand Prix","Miami Grand Prix","Monaco Grand Prix","Saudi Arabian Grand Prix","Singapore Grand Prix","Spanish Grand Prix","São Paulo Grand Prix","United States Grand Prix"]
#2021
# race_multi=["Abu Dhabi Grand Prix","Austrian Grand Prix","Azerbaijan Grand Prix","Bahrain Grand Prix","Belgian Grand Prix","British Grand Prix","Dutch Grand Prix","Emilia Romagna Grand Prix","French Grand Prix","Hungarian Grand Prix","Italian Grand Prix","Mexico City Grand Prix","Monaco Grand Prix","Portuguese Grand Prix","Qatar Grand Prix","Russian Grand Prix","Saudi Arabian Grand Prix","Spanish Grand Prix","Styrian Grand Prix","São Paulo Grand Prix","Turkish Grand Prix","United States Grand Prix"]
#2020
race_multi=["70th Anniversary Grand Prix","Abu Dhabi Grand Prix","Austrian Grand Prix","Bahrain Grand Prix","Belgian Grand Prix","British Grand Prix","Eifel Grand Prix","Emilia Romagna Grand Prix","Hungarian Grand Prix","Italian Grand Prix","Portuguese Grand Prix","Russian Grand Prix","Sakhir Grand Prix","Spanish Grand Prix","Styrian Grand Prix","Turkish Grand Prix","Tuscan Grand Prix"]

year_multi=2020
for i in race_multi:    

    GP=df_bq[((df_bq["GP"]==i) & (df_bq["Year"]==year_multi))]
    GP["LapNumber"]=GP["LapNumber"].astype(int)
    GP=GP.sort_values(['Driver','LapNumber'])
    GP["DriverNumber"]=GP["DriverNumber"].astype(int)
    year=GP['Year'].iloc[0]
    gp=GP['GP'].iloc[0]
    GP.head()
    fig=px.line(GP, x="LapNumber", y="Position", color='Driver',
            title="Race Position Ladder for the {} {}".format(year_multi,i),
            hover_data=['Compound','TyreLife','Team','LapTime','Stint', 'PitInTime', 'PitOutTime', 'Sector1Time', 'Sector2Time', 'LapStartTime', 'Year'],
            height=1000, width=1200, template="plotly_dark",
                color_discrete_map={
                 "ALB": "#64C4FF",
                 "ALO": "#229971",
                 "BOT": "#52e252",
                 "COL": "#37BEDD",
                 "GAS": "#0093cc",
                 "HAM": "#27F4D2",
                 "HUL": "#B6BABD",
                 "LEC": "#E80020",
                 "MAG": "#B6BABD",
                 "NOR": "#FF8000",
                 "OCO": "#0093cc",
                 "PER": "#3671C6",
                 "PIA": "#FF8000",
                 "RIC": "#6692FF",
                 "RUS": "#27F4D2",
                 "SAI": "#E80020",
                 "SAR": "#37BEDD",
                 "STR": "#229971",
                 "TSU": "#6692FF",
                 "VER": "#3671C6",
                 "ZHO": "#52e252"
            },
            )

    fig.update_layout(
        title_x=0.5,
        yaxis_range=[1, 21],
        margin=dict(l=10, r=5, t=55, b=20),
        font=dict(
            family="PT Sans Narrow",
            size=13,
            color="White"
        ),

        yaxis = dict(
        tickmode = 'linear',
        tick0 = 1,
        dtick = 1
        ),

        hoverlabel=dict(
            bgcolor="white",
            font_size=16,
            font_family="PT Sans Narrow"
        ),

        margin_pad=10
    )
    fig.update_yaxes(autorange="reversed")
    fig.write_html("/Users/rdesh723/statpulse-html/plots/Race Ladder/{}/{} {} Race Ladder.html".format(year_multi,year_multi,i),full_html=False, include_plotlyjs='cdn')

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_6284/3949501130.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_6284/3949501130.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_6284/3949501130.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

In [ ]:
#LAP TIMES

In [15]:
#Convert LapTime to Time Delta for plotting on Y Axis
GP['LapTime_TD']= pd.to_timedelta(GP["LapTime"])
GP["LapTime"]=GP['LapTime'].str.split('days ').str[1]
GP.loc[:, "LapTime (s)"] = GP["LapTime_TD"].dt.total_seconds()
GP=GP[GP["LapTime (s)"]<500]
GP.head()

,Time,Driver,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,Sector3Time,Sector1SessionTime,Sector2SessionTime,Sector3SessionTime,SpeedI1,SpeedI2,SpeedFL,SpeedST,IsPersonalBest,Compound,TyreLife,FreshTyre,Team,LapStartTime,LapStartDate,TrackStatus,Position,Deleted,DeletedReason,FastF1Generated,IsAccurate,Year,GP,LapTime_TD,LapTime (s)
21467,0 days 00:59:02.548000,ALB,23,00:01:51.269000,1,1.0,NaT,NaT,NaT,0 days 00:00:41.668000,0 days 00:00:35.637000,NaT,0 days 00:58:26.960000,0 days 00:59:02.712000,199.0,179.0,197.0,313.0,False,MEDIUM,1.0,True,Williams,0 days 00:57:10.953000,NaT,12,18.0,False,,False,False,2024,United States Grand Prix,0 days 00:01:51.269000,111.269
20517,0 days 01:00:47.092000,ALB,23,00:01:44.544000,2,1.0,NaT,NaT,0 days 00:00:28.425000,0 days 00:00:41.319000,0 days 00:00:34.800000,0 days 00:59:30.973000,0 days 01:00:12.292000,0 days 01:00:47.092000,196.0,172.0,197.0,312.0,True,MEDIUM,2.0,True,Williams,0 days 00:59:02.548000,NaT,12,18.0,False,,False,True,2024,United States Grand Prix,0 days 00:01:44.544000,104.544
21015,0 days 01:03:03.790000,ALB,23,00:02:16.698000,3,1.0,NaT,0 days 01:03:02.808000,0 days 00:00:27.948000,0 days 00:01:03.889000,0 days 00:00:44.861000,0 days 01:01:15.040000,0 days 01:02:18.929000,0 days 01:03:03.790000,208.0,138.0,NaN,120.0,False,MEDIUM,3.0,True,Williams,0 days 01:00:47.092000,NaT,24,17.0,False,,False,False,2024,United States Grand Prix,0 days 00:02:16.698000,136.698
21017,0 days 01:08:01.846000,ALB,23,00:02:21.937000,5,2.0,NaT,NaT,0 days 00:00:34.174000,0 days 00:00:58.736000,0 days 00:00:49.027000,0 days 01:06:14.083000,0 days 01:07:12.819000,0 days 01:08:01.846000,199.0,158.0,204.0,125.0,False,MEDIUM,2.0,True,Williams,0 days 01:05:39.909000,NaT,41,19.0,False,,False,False,2024,United States Grand Prix,0 days 00:02:21.937000,141.937
21018,0 days 01:09:46.498000,ALB,23,00:01:44.652000,6,2.0,NaT,NaT,0 days 00:00:28.083000,0 days 00:00:41.425000,0 days 00:00:35.144000,0 days 01:08:29.929000,0 days 01:09:11.354000,0 days 01:09:46.498000,198.0,177.0,204.0,316.0,False,MEDIUM,3.0,True,Williams,0 days 01:08:01.846000,NaT,1,19.0,False,,False,True,2024,United States Grand Prix,0 days 00:01:44.652000,104.652


In [16]:
fig_all_laps=px.scatter(GP[GP["LapTime (s)"]<GP['LapTime (s)'].min()*1.1], 
        x="LapNumber", 
        y="LapTime (s)", 
        color='Driver',
        title="All Lap Times for the {} {}".format(year,gp),
        hover_data=['Compound','TyreLife','Team','LapTime','Stint', 'PitInTime', 'PitOutTime', 'Sector1Time', 'Sector2Time', 'LapStartTime', 'Year'],
        height=700,
          width=1200, 
        template="plotly_dark",
        color_discrete_map={
                 "ALB": "#64C4FF",
                 "ALO": "#229971",
                 "BEA": "#B6BABD",
                 "BOT": "#52e252",
                 "COL": "#37BEDD",
                 "GAS": "#0093cc",
                 "HAM": "#27F4D2",
                 "HUL": "#B6BABD",
                 "LEC": "#E80020",
                 "MAG": "#B6BABD",
                 "NOR": "#FF8000",
                 "OCO": "#0093cc",
                 "PER": "#3671C6",
                 "PIA": "#FF8000",
                 "RIC": "#6692FF",
                 "RUS": "#27F4D2",
                 "SAI": "#E80020",
                 "SAR": "#37BEDD",
                 "STR": "#229971",
                 "TSU": "#6692FF",
                 "VER": "#3671C6",
                 "ZHO": "#52e252"
          },
        )
fig_all_laps.update_layout(
    title_x=0.5,
    hoverlabel=dict(
    bgcolor="white",
    font_size=16,
    font_family="PT Sans Narrow",
    # margin=dict(l=5, r=5, t=35, b=5),
    ),
    updatemenus=[
        {
            "buttons": [
                {
                    "label": t.name,
                    "method": "restyle",
                    "args": [{"visible": [t2.name == t.name for t2 in fig_all_laps.data]}],
                }
                for t in fig_all_laps.data
            ]
        }
    ],
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="White"
    ),

    yaxis = dict(
    tickmode = 'linear',
    tick0 = 0,
    dtick = 2
    ),

    margin_pad=10
)

In [17]:
fig_all_laps.write_html("/Users/rdesh723/statpulse-html/plots/Race Lap Times/{}/{} {} All Lap Times.html".format(year,year,gp),full_html=False, include_plotlyjs='cdn')

In [ ]:
#Multi GP Lap Times

In [25]:
#2024
# race_multi=["Bahrain Grand Prix", "Saudi Arabian Grand Prix", "Australian Grand Prix", "Japanese Grand Prix","Chinese Grand Prix","Miami Grand Prix","Emilia Romagna Grand Prix","Monaco Grand Prix","Canadian Grand Prix","Spanish Grand Prix","Austrian Grand Prix","British Grand Prix","Hungarian Grand Prix","Belgian Grand Prix"]
#2023
# race_multi=["Abu Dhabi Grand Prix","Australian Grand Prix","Austrian Grand Prix","Azerbaijan Grand Prix","Bahrain Grand Prix","Belgian Grand Prix","British Grand Prix","Canadian Grand Prix","Dutch Grand Prix","Hungarian Grand Prix","Italian Grand Prix","Japanese Grand Prix","Las Vegas Grand Prix","Mexico City Grand Prix","Miami Grand Prix","Monaco Grand Prix","Pre-Season Testing","Qatar Grand Prix","Saudi Arabian Grand Prix","Singapore Grand Prix","Spanish Grand Prix","São Paulo Grand Prix","United States Grand Prix"]
# 2022
# race_multi=["Abu Dhabi Grand Prix","Australian Grand Prix","Austrian Grand Prix","Azerbaijan Grand Prix","Bahrain Grand Prix","Belgian Grand Prix","British Grand Prix","Canadian Grand Prix","Dutch Grand Prix","Emilia Romagna Grand Prix","French Grand Prix","Hungarian Grand Prix","Italian Grand Prix","Japanese Grand Prix","Mexico City Grand Prix","Miami Grand Prix","Monaco Grand Prix","Saudi Arabian Grand Prix","Singapore Grand Prix","Spanish Grand Prix","São Paulo Grand Prix","United States Grand Prix"]
#2021
# race_multi=["Abu Dhabi Grand Prix","Austrian Grand Prix","Azerbaijan Grand Prix","Bahrain Grand Prix","Belgian Grand Prix","British Grand Prix","Dutch Grand Prix","Emilia Romagna Grand Prix","French Grand Prix","Hungarian Grand Prix","Italian Grand Prix","Mexico City Grand Prix","Monaco Grand Prix","Portuguese Grand Prix","Qatar Grand Prix","Russian Grand Prix","Saudi Arabian Grand Prix","Spanish Grand Prix","Styrian Grand Prix","São Paulo Grand Prix","Turkish Grand Prix","United States Grand Prix"]
#2020
race_multi=["70th Anniversary Grand Prix","Abu Dhabi Grand Prix","Austrian Grand Prix","Bahrain Grand Prix","Belgian Grand Prix","British Grand Prix","Eifel Grand Prix","Emilia Romagna Grand Prix","Hungarian Grand Prix","Italian Grand Prix","Portuguese Grand Prix","Russian Grand Prix","Sakhir Grand Prix","Spanish Grand Prix","Styrian Grand Prix","Turkish Grand Prix","Tuscan Grand Prix"]

year_multi=2020
for i in race_multi:
    GP=df_bq[((df_bq["GP"]==i) & (df_bq["Year"]==year_multi))]
    GP["LapNumber"]=GP["LapNumber"].astype(int)
    GP=GP.sort_values(['Driver','LapNumber'])
    GP["DriverNumber"]=GP["DriverNumber"].astype(int)
    year=GP['Year'].iloc[0]
    gp=GP['GP'].iloc[0]
    GP['LapTime_TD']= pd.to_timedelta(GP["LapTime"])
    GP["LapTime"]=GP['LapTime'].str.split('days ').str[1]
    GP.loc[:, "LapTime (s)"] = GP["LapTime_TD"].dt.total_seconds()
    GP=GP[GP["LapTime (s)"]<500]
    fig_all_laps=px.scatter(GP[GP["LapTime (s)"]<GP['LapTime (s)'].min()*1.15], 
            x="LapNumber", 
            y="LapTime (s)", 
            color='Driver',
            title="All Lap Times for the {} {}".format(year_multi,i),
            hover_data=['Compound','TyreLife','Team','LapTime','Stint', 'PitInTime', 'PitOutTime', 'Sector1Time', 'Sector2Time', 'LapStartTime', 'Year'],
            height=700,
            width=1200, 
            template="plotly_dark",
            color_discrete_map={
                 "ALB": "#64C4FF",
                 "ALO": "#229971",
                 "BOT": "#52e252",
                 "COL": "#37BEDD",
                 "GAS": "#0093cc",
                 "HAM": "#27F4D2",
                 "HUL": "#B6BABD",
                 "LEC": "#E80020",
                 "MAG": "#B6BABD",
                 "NOR": "#FF8000",
                 "OCO": "#0093cc",
                 "PER": "#3671C6",
                 "PIA": "#FF8000",
                 "RIC": "#6692FF",
                 "RUS": "#27F4D2",
                 "SAI": "#E80020",
                 "SAR": "#37BEDD",
                 "STR": "#229971",
                 "TSU": "#6692FF",
                 "VER": "#3671C6",
                 "ZHO": "#52e252"
            },
            )
    fig_all_laps.update_layout(
        title_x=0.5,
        hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow",
        ),
        updatemenus=[
            {
                "buttons": [
                    {
                        "label": t.name,
                        "method": "restyle",
                        "args": [{"visible": [t2.name == t.name for t2 in fig_all_laps.data]}],
                    }
                    for t in fig_all_laps.data
                ]
            }
        ],
        font=dict(
            family="PT Sans Narrow",
            size=14,
            color="White"
        ),

        yaxis = dict(
        tickmode = 'linear',
        tick0 = 0,
        dtick = 2
        ),

        margin_pad=10
    )
    fig_all_laps.write_html("/Users/rdesh723/statpulse-html/plots/Race Lap Times/{}/{} {} All Lap Times.html".format(year_multi,year_multi,i),full_html=False, include_plotlyjs='cdn')

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_7503/1244651401.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_7503/1244651401.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_7503/1244651401.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

In [10]:
# GP=GP.sort_values(['LapNumber', "Driver"])
# GP.tail()

In [11]:
# fig2=px.scatter(GP,
#         x="LapNumber",
#         y="Position",
#         color='DriverNumber',
#         animation_frame="LapNumber",
#         title="Race Position Ladder for the {} {}".format(year,gp),
#         # animation_group='LapNumber',
#         range_x=[0,len(GP['LapNumber'].unique())+1],
#         text='Driver',
#         range_y=[0,20],
#         hover_data=['Compound','TyreLife','Team','LapTime','Stint', 'PitInTime', 'PitOutTime', 'Sector1Time', 'Sector2Time', 'LapStartTime', 'Year'],
#         height=1000, width=1800, template="plotly_dark",
#            color_discrete_map={
#                  "ALB": "#64C4FF",
#                  "ALO": "#229971",
#                  "BOT": "#52e252",
#                  "GAS": "#0093cc",
#                  "HAM": "#27F4D2",
#                  "HUL": "#B6BABD",
#                  "LEC": "#E80020",
#                  "MAG": "#B6BABD",
#                  "NOR": "#FF8000",
#                  "OCO": "#0093cc",
#                  "PER": "#3671C6",
#                  "PIA": "#FF8000",
#                  "RIC": "#6692FF",
#                  "RUS": "#27F4D2",
#                  "SAI": "#E80020",
#                  "SAR": "#37BEDD",
#                  "STR": "#229971",
#                  "TSU": "#6692FF",
#                  "VER": "#3671C6",
#                  "ZHO": "#52e252"
#           },
#          )


# fig2.update_yaxes(autorange="reversed")

# fig.update_traces(textposition='middle right')

# fig2.update_layout(
#     title_x=0.5,
#     hoverlabel=dict(
#     bgcolor="white",
#     font_size=16,
#     font_family="PT Sans Narrow"
#     ),
#     font=dict(
#         family="PT Sans Narrow",
#         size=14,
#         color="White"
#     ),

#     yaxis = dict(
#     tickmode = 'linear',
#     tick0 = 0,
#     dtick = 1
#     ),

#     margin_pad=10
# )

# fig2.update(layout_coloraxis_showscale=False)
# # for x,y in zip(GP[GP["Driver"]=="SAI"].Driver, GP[GP["Driver"]=="SAI"].Position):
# #   for png in (Path.cwd().joinpath("f1").glob("*.png")):
# #     if str.split(str.split(str(png),".")[0],"/")[3]==x:
# #       image=str(png)
# #       fig2.add_layout_image(
# #           x=x,
# #           y=y+4,
# #           source=Image.open(image),
# #           xref="x",
# #           yref="y",
# #           sizex=5,
# #           sizey=5,
# #           xanchor="center",
# #           yanchor="middle",
# #       )
# fig2.show()

In [12]:
# fig2.write_html("{} {} Race Animation.html".format(year,gp))

# Fastest Driver in each lap

In [86]:
GP_Rk=GP[GP["LapTime"]!="NaT"]
GP_Rk["RK"] = GP_Rk.groupby("LapNumber")["LapTime"].rank(method="dense", ascending=True)
GP_FASTEST_LAPS=GP_Rk[GP_Rk["RK"]==1.0].sort_values("LapNumber")
GP_FASTEST_LAPS=GP_FASTEST_LAPS[["Time","Driver","Team","LapTime","LapNumber","Stint","PitOutTime","PitInTime","Sector1Time","Sector2Time","Sector3Time","SpeedI1","SpeedI2","TyreLife","TrackStatus","Position","Year","GP","RK"]]
GP_FASTEST_LAPS.head()

,Time,Driver,Team,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,Sector3Time,SpeedI1,SpeedI2,TyreLife,TrackStatus,Position,Year,GP,RK
543798,0 days 00:58:40.015000,RUS,Mercedes,00:01:43.723000,1,1.0,NaT,NaT,NaT,0 days 00:00:31.645000,0 days 00:00:37.671000,190.0,264.0,1.0,12,1.0,2024,Canadian Grand Prix,1.0
108881,0 days 01:00:16.571000,RUS,Mercedes,00:01:36.556000,2,1.0,NaT,NaT,0 days 00:00:28.287000,0 days 00:00:30.597000,0 days 00:00:37.672000,198.0,264.0,2.0,1,1.0,2024,Canadian Grand Prix,1.0
74979,0 days 01:02:01.046000,MAG,Haas F1 Team,00:01:36.134000,3,1.0,NaT,NaT,0 days 00:00:27.936000,0 days 00:00:31.214000,0 days 00:00:36.984000,198.0,268.0,3.0,1,4.0,2024,Canadian Grand Prix,1.0
364215,0 days 01:03:35.039000,MAG,Haas F1 Team,00:01:33.993000,4,1.0,NaT,NaT,0 days 00:00:27.334000,0 days 00:00:29.992000,0 days 00:00:36.667000,215.0,265.0,4.0,1,4.0,2024,Canadian Grand Prix,1.0
129219,0 days 01:05:04.364000,VER,Red Bull Racing,00:01:33.859000,5,1.0,NaT,NaT,0 days 00:00:27.574000,0 days 00:00:30.293000,0 days 00:00:35.992000,200.0,267.0,5.0,1,2.0,2024,Canadian Grand Prix,1.0
